# Locating wifi beacons
Logan Gall, gall0487

13 Dec. 2023

This file runs a short script that reads in pre-scanned wifi beacons (colelcted from Locate_APs.ipynb), and then uses trilateration to get approximate locations of each beacon based on MAC address. This allows for us to use these known locations for future trilateration based on a wifi scan.

In [ ]:
import csv
from scipy.optimize import least_squares

In [1]:
# Function to read WiFi data from a CSV file
def read_wifi_data(filename="wifi_scan_results.csv"):
    with open(filename, 'r') as csvfile:
        # Using a CSV DictReader to read the file into a dictionary format
        reader = csv.DictReader(csvfile)
        # Converting the read data into a list of dictionaries
        wifi_data = list(reader)
    return wifi_data

# Function to organize WiFi data by BSSID
def organize_data_by_bssid(wifi_data):
    bssid_data = {}  # Dictionary to hold data organized by BSSID
    for entry in wifi_data:
        bssid = entry['BSSID']
        # Initialize a new entry for a BSSID if it's not already present
        if bssid not in bssid_data:
            bssid_data[bssid] = {'coordinates': [], 'signals': []}
        # Convert coordinates and signal strengths to appropriate data types
        coords = [float(entry['X Coordinate']), float(entry['Y Coordinate']), float(entry['Z Coordinate'])]
        signal = abs(int(entry['Signal']))  # Convert signal strength to a positive value
        # Append the coordinates and signal strength to the respective BSSID entry
        bssid_data[bssid]['coordinates'].append(coords)
        bssid_data[bssid]['signals'].append(signal)
    return bssid_data

# Function to perform trilateration
def trilateration(nodes, distances):
    # Nested function defining the equations for trilateration
    def equations(point):
        x, y, z = point
        # Compute the squared differences between measured and calculated distances
        return [((x - nx)**2 + (y - ny)**2 + (z - nz)**2 - d**2) for [nx, ny, nz], d in zip(nodes, distances)]
    # Starting with an initial guess of [0, 0, 0]
    initial_guess = [0, 0, 0]
    # Using least squares to solve the equations
    result = least_squares(equations, initial_guess)
    return result.x

In [ ]:
# Calling the function to read the data
wifi_data = read_wifi_data()

# Organizing the data by BSSID (MAC Address)
bssid_data = organize_data_by_bssid(wifi_data)

# Dictionary to store estimated positions
estimated_positions = {}
for bssid, data in bssid_data.items():
    # Ensure there are at least 3 points for accurate trilateration
    if len(data['coordinates']) >= 3:
        position = trilateration(data['coordinates'], data['signals'])
        estimated_positions[bssid] = position

# Writing the estimated positions to a new CSV file
with open('estimated_positions.csv', 'w', newline='') as csvfile:
    fieldnames = ['BSSID', 'X Position', 'Y Position', 'Z Position']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for bssid, position in estimated_positions.items():
        writer.writerow({'BSSID': bssid, 'X Position': position[0], 'Y Position': position[1], 'Z Position': position[2]})